In [49]:
#### Import packages and set working directory
import pandas as pd
import os
from google.colab import drive
import random
import re
import math
import numpy as np

drive.mount('/content/drive/')
os.getcwd()
os.chdir('/content/drive/My Drive/cse517/hw2')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#############################################
########### Data Preprocessing ##############
#############################################
#### Function which reads in json file 
import json
def read_dataset(filename):
	with open(filename,'r') as lines:
		dataset = [json.loads(line) for line in lines]
	lines.close()
	return dataset

In [0]:
### Import training set, dev set, and test set
train = read_dataset("twt.train.json")
dev = read_dataset("twt.dev.json")
test = read_dataset("twt.test.json")

In [0]:
### Add START and STOP in each sentence
SENTENCE_END = "</s>"
SENTENCE_START = ["<s>", 'START_1']
SENTENCE_START_2 = ["<ss>", 'START_2']
for i in range(len(train)):
  train[i].insert(0, SENTENCE_START)
  train[i].insert(0, SENTENCE_START_2)
  train[i].append(['</s>', 'STOP'])
  
for i in range(len(dev)):
  dev[i].insert(0, SENTENCE_START)
  dev[i].insert(0, SENTENCE_START_2)
  dev[i].append(['</s>', 'STOP'])
  
for i in range(len(test)):
  test[i].insert(0, SENTENCE_START)
  test[i].insert(0, SENTENCE_START_2)
  test[i].append(['</s>', 'STOP'])

In [0]:
### get Unigram state and bigram state 
unigram_word = dict()
bigram_state = dict()
unigram_state = dict()
#unigram_emission = dict()
for sentence in train:
  previous_state = None
  for word, state in sentence:
    unigram_state[state] = unigram_state.get(state, 0) + 1
    unigram_word[word] = unigram_word.get(word, 0) +1
    #unigram_emission[(word, state)] = unigram_emission.get((word, state), 0 ) +1
    if previous_state != None and previous_state != 'STOP':
      bigram_state[(previous_state, state)] = bigram_state.get((previous_state, state), 0) +1
    previous_state = state
      

In [0]:
### Obtain the low frequent words 
unigram_word_copy = dict(unigram_word)
low_freq =5 
low_freq_word ={}
for (key, value) in unigram_word_copy.items():
  if value <= low_freq:
    low_freq_word[key] = value
    del unigram_word[key]

present_word = unigram_word.keys()  


In [0]:
#############################################
################ OOV Rules #################
#############################################

### Convert low frequent words into different classes based on different rules
def get_type(word):
  if "#" in word:
    return("hash")
  elif "@" in word:
    return("R@")
  elif "https" in word:
    return("link##")
  elif any(i.isdigit() for i in word):
    return("number$%")
  elif ":" in word:
    return(":word")
  else:
    return("<UNK>")

In [0]:
### Convert the low frequent words into different classes and update the unigram 
train_low_freq_word = {}
for i in low_freq_word.keys():
  new_type = get_type(i)
  train_low_freq_word[new_type] = train_low_freq_word.get(new_type, 0) + low_freq_word[i] 

unigram_word.update(train_low_freq_word)

In [0]:
### Construct unigram emission probability 
unigram_emission = dict()
for sentence in train:
  for word, state in sentence:
    if word not in present_word:
      new_type = get_type(word)
      unigram_emission[(new_type, state)] = unigram_emission.get((new_type, state), 0 ) +1
    else:
      unigram_emission[(word, state)] = unigram_emission.get((word, state), 0 ) +1
     

In [0]:
for i in range(len(train)):
  for j in range(len(train[i])):
    if train[i][j][0] not in present_word:
      train[i][j][0] = get_type(train[i][j][0])

In [0]:
### Convert low frequent words in dev set
for i in range(len(dev)):
  for j in range(len(dev[i])):
    if dev[i][j][0] not in present_word:
      dev[i][j][0] = get_type(dev[i][j][0])

In [0]:
### Bigram HMM probability for states
unique_state_length = len(unigram_state.keys()) -3 
def calculate_bigramHMM_probability_LP(previous_state, state):
  if unigram_state.get(state, 0) ==0:
    return(1/unique_state_length)
  else:
    bigram_state_numerator = bigram_state.get((previous_state, state), 0) 
    bigram_state_denominator = unigram_state.get(previous_state) 
    return(bigram_state_numerator/bigram_state_denominator)

In [0]:
### Unigram HMM probability for state
unigram_state_corpus = sum(unigram_state.values()) - unigram_state['STOP'] - unigram_state['START_1'] -  unigram_state['START_2']
def calculate_unigramHMM_probability_LP(state):
    unigram_state_numerator = unigram_state.get(state, 0) 
    unigram_state_denominator = unigram_state_corpus
    return(unigram_state_numerator/unigram_state_denominator)

In [0]:
### Linear interpolation smooth 
def calculate_interpolationHMM_probability(previous_state, state):
  bi_prob = calculate_bigramHMM_probability_LP(previous_state, state)
  uni_prob = calculate_unigramHMM_probability_LP(state)
  res = lambda_1 * bi_prob + lambda_2 * uni_prob 
  return(res)

In [0]:
### Emission probability 
def calculate_emissionHMM_probability_LP(word, state):
  emission_numerator = unigram_emission.get((word, state),0)
  emission_denominator = unigram_state[state]
  return(emission_numerator/emission_denominator)

In [0]:
###################################################
########### Bigram Viterbi algorithm ##############
##################################################
### Viterbi algorithm to get the pi matrix 
lambda_1 = 0.6
lambda_2 = 0.4
unique_state = unigram_state.keys()
def viterbi_deconding(text):
  V= [{}]
  prev = None
  for state in unique_state:
    if state == "START_1":
      V[0][state] = {"prob":1, "prev":None}
    else:
      V[0][state] = {"prob":0, "prev":None}
  
  for i in range(1, len(text)):
    V.append({})
    for state in unique_state:
      V[i][state] = {"prob":0, "prev":None}
    for state in unique_state:
      max_prob = 0
      for prev in unique_state:
        prob = V[i-1][prev]['prob'] * calculate_emissionHMM_probability_LP(text[i][0], state) * calculate_interpolationHMM_probability(prev, state)
        if prob > max_prob:
          V[i][state] = {"prob": prob, "prev":prev}
          max_prob = prob          
  return(V)

In [0]:
### Decode the state result from the Pi matrix 
def viterbi_result(text):
  V = viterbi_deconding(text)
  opt = []
  prev= None
  max_prob = max(value["prob"] for value in V[-1].values())
  for st, data in V[-1].items():
    if data["prob"] == max_prob:
      opt.append(st)
      previous = V[-1][st]['prev']
      break
  for t in range(len(V) - 2, -1, -1):
    opt.insert(0, previous)
    previous = V[t][previous]["prev"]
  return(opt)

In [0]:
### Access training accuracy
accuracy_train = 0
word_train = 0
start_count =0 
for text in train:
  pred = viterbi_result(text) 
  start_count +=1 
  for i in range(len(pred)):
    word_train  += 1
    if text[i][1] == pred[i]:
      accuracy_train += 1

In [68]:
(accuracy_train-start_count*3)/(word_train-start_count*3)

0.8367236476996335

In [0]:
### Access dev accuracy and confusion matrix
confusion_mat = np.zeros((len(unique_state), len(unique_state)))
confusion_res = pd.DataFrame(confusion_mat, columns=unique_state, index=unique_state)
accuracy = 0
word = 0

In [0]:
for text in dev:
  pred = viterbi_result(text) 
  for i in range(len(pred)):
    word  += 1
    #confusion_res[text[i][1]][pred[i]] += 1
    if text[i][1] == pred[i]:
      accuracy += 1

In [71]:
(accuracy - 5000*3)/(word- 5000*3)

0.8300017689722271

In [0]:
### Convert low frequent words in test set
for i in range(len(test)):
  for j in range(len(test[i])):
    if test[i][j][0] not in present_word:
      test[i][j][0] = get_type(test[i][j][0])
confusion_mat_test = np.zeros((len(unique_state), len(unique_state)))
confusion_res_test = pd.DataFrame(confusion_mat_test, columns=unique_state, index=unique_state)
accuracy_test = 0
word_test = 0


In [0]:
### Access test accuracy
for text in test:
  pred = viterbi_result(text) 
  for i in range(len(pred)):
    word_test  += 1
    if text[i][1] == pred[i]:
      accuracy_test += 1

In [74]:
(accuracy_test - 5000*3)/(word_test-5000*3)

0.8312908885654829

In [75]:
len(test)

5000

In [0]:
## Output the mismatches and their counts 
wrong_list ={}
for text in dev:
  pred = viterbi_result(text) 
  for i in range(len(pred)):
    #word  += 1
    #confusion_res[text[i][1]][pred[i]] += 1
    if text[i][1] != pred[i]:
      wrong_list[(text[i][1], pred[i])] = wrong_list.get((text[i][1], pred[i]),0) +1

In [0]:
###################################################
########### Trigram Viterbi algorithm ##############
##################################################


In [0]:
#### get trigram state transition probabilities
trigram_state = dict()
for sentence in train:
  state_1 = None
  state_2 = None
  for word, state in sentence:
    if state_1 != None and state_2 != None:
      trigram_state[(state_1, state_2, state)] = trigram_state.get((state_1, state_2, state), 0) +1
    state_1 = state_2
    state_2 = state

In [0]:
def calculate_trigramHMM_probability_LP(state_1, state_2, state):
  if bigram_state.get((state_1, state_2), 0) ==0:
    return(1/unique_state_length)
  else:
    trigram_state_numerator = trigram_state.get((state_1, state_2, state), 0) 
    trigram_state_denominator = bigram_state.get((state_1, state_2), 0)  
    return(trigram_state_numerator/trigram_state_denominator)

In [0]:
def calculate_trigram_interpolationHMM_probability(state_1, state_2, state):
  tri_prob = calculate_trigramHMM_probability_LP(state_1, state_2, state)
  bi_prob = calculate_bigramHMM_probability_LP(state_2, state)
  uni_prob = calculate_unigramHMM_probability_LP(state)
  res = lambda_1 * tri_prob + lambda_2 * bi_prob + lambda_3 * uni_prob 
  return(res)


In [0]:
lambda_1 = 0.4
lambda_2 = 0.3
lambda_3 = 0.3
unique_state = unigram_state.keys()
def viterbi_deconding_trigram(text):
  V= [{}]
  state_1 = None
  state_2 = None
  for state in unique_state:
    if state == "START_1":
      V[0][state] = {"prob":1, "state_1":None, "state_2":None}
    else:
      V[0][state] = {"prob":0, "state_1":None, "state_2":None}
  for i in range(1, len(text)):
    V.append({})
    for state in unique_state:
      V[i][state] = {"prob":0, "state_1":None, "state_2":None}
    for state in unique_state:
      max_prob = 0
      for state_1 in unique_state:
        for state_2 in unique_state:
          prob = V[i-1][state_2]['prob'] * calculate_emissionHMM_probability_LP(text[i][0], state) * calculate_trigram_interpolationHMM_probability(state_1, state_2, state)
          if prob > max_prob:
            V[i][state] = {"prob": prob, "state_1":state_1, "state_2":state_2}
            max_prob = prob          
  return(V)

In [0]:
def viterbi_result_trigram(text):
  V = viterbi_deconding_trigram(text)
  opt = []
  state_1 = None
  state_2 = None
  max_prob = max(value["prob"] for value in V[-1].values())
  for st, data in V[-1].items():
    if data["prob"] == max_prob:
      opt.append(st)
      state_1 = V[-1][st]['state_1']
      state_2 = V[-1][st]['state_2']
      break
  for t in range(len(V) - 2, -1, -1):
    opt.insert(0, state_2)
    state_2 = V[t][state_2]["state_2"]
  return(opt)

In [0]:
#viterbi_result_trigram(train[0])

In [0]:
accuracy_train_tri = 0 
word_train = 0 
for text in train:
  pred = viterbi_result_trigram(text) 
  for i in range(len(pred)):
    word_train += 1
    if text[i][1] == pred[i]:
      accuracy_train_tri +=1

In [0]:
accuracy_tri = 0
word = 0
for text in dev:
  pred = viterbi_result_trigram(text) 
  for i in range(len(pred)):
    word += 1
    if text[i][1] == pred[i]:
      accuracy_tri +=1